In [1]:
################## IMPORT LIBRARIES ##################

import numpy as np
import sys
import importlib
import random 
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (4, 3)
import torch 

In [2]:
################## IMPORT MY MODULES ##################
sys.path.append('../src')

import helpers as hlp
import evaluation
import dataset as ds
import trainer
import models

importlib.reload(evaluation)
importlib.reload(hlp)
importlib.reload(ds)
importlib.reload(trainer)
importlib.reload(models)

<module 'models' from '/home/ubuntu/guestxr2/home/ubuntu/joanna/CWUNET/notebooks/../src/models.py'>

In [6]:
################## LOAD TRAINING RESULTS AND CONFIG  ##################

datapath="/home/ubuntu/guestxr2/home/ubuntu/joanna/CWUNET/results/"
exp_tag=""
train_tag="big_1"

# load the best checkpoint from the training
config ,train_results = trainer.load_train_results(datapath, exp_tag, train_tag,configtype="yaml") 


In [7]:
################## LOAD DATASET ##################

# instantiate a test data set 
config["df_metadata"]="/home/ubuntu/guestxr2/home/ubuntu/joanna/CWUNET/dataset-metadata/ds1_metadata_big.csv"
config["split"]="test"
config["p_noise"]=0
dataset=ds.DatasetReverbTransfer(config)

In [8]:
################## LOAD MODELS AND TRAINING WEIGHTS  ##################

model=trainer.load_chosen_model(config,config["modeltype"])
model.load_state_dict(train_results["model_state_dict"])


<All keys matched successfully>

In [23]:
################## GENERATE EMBEDDING ARRAY AND PARAMETER ARRAYS FROM MULTIPLE DATA POINTS ##################

Nr_embeddings=100
emb_arr=np.zeros([Nr_embeddings,512])
rt30_meas_arr=np.zeros([Nr_embeddings,1])
edt_meas_arr=np.zeros([Nr_embeddings,1])
c50_meas_arr=np.zeros([Nr_embeddings,1])
rt60_set_arr=np.zeros([Nr_embeddings,1])
rt60_true_arr=np.zeros([Nr_embeddings,1])

for j in range(0,Nr_embeddings):
    data=dataset[j]
    # get style audio
    sStyle_in=data[1]
    # get style info
    df_Style=dataset.get_info(j,id="style")
    # get style embedding of size 512
    with torch.no_grad():
        embedding=model.conditioning_network(sStyle_in.unsqueeze(0).to(config["device"]))
        emb_arr[j,:]=embedding.cpu().numpy()
    rt30_meas_arr[j]=df_Style["rt30_meas"]
    rt60_set_arr[j]=df_Style["rt60_set"]
    rt60_true_arr[j]=df_Style["rt60_true"]
    edt_meas_arr[j]=df_Style["edt_meas"]
    c50_meas_arr[j]=df_Style["c50_meas"]
    if j==Nr_embeddings:
        break
    

In [ ]:

# Save arrays with embeddings and parameters
np.save('emb_arr3.npy', emb_arr)
np.save('rt30_meas_arr3.npy', rt30_meas_arr)
np.save('rt60_set_arr3.npy', rt60_set_arr)
np.save('rt60_true_arr3.npy', rt60_true_arr)
np.save('edt_meas_arr3.npy', edt_meas_arr)
np.save('c50_meas_arr3.npy', c50_meas_arr)

In [ ]:
# Read the saved arrays
emb_arr = np.load('emb_arr3.npy')
rt30_meas_arr = np.load('rt30_meas_arr3.npy')
rt60_set_arr = np.load('rt60_set_arr3.npy')
rt60_true_arr = np.load('rt60_true_arr3.npy')
edt_meas_arr = np.load('edt_meas_arr3.npy')
c50_meas_arr = np.load('c50_meas_arr3.npy')

In [30]:
################## PERFORM DIMENSIONALITY REDUCTION ON THE EMBEDDING SPACE ##################

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import umap

indices_subset=range(0,Nr_embeddings)
# using pca:
embeddings_pca=PCA(n_components=2).fit_transform(emb_arr[indices_subset,:])
# using first pca and then tsne:
embeddings_pca_tsne = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=50).fit_transform(PCA(n_components=20).fit_transform(emb_arr[indices_subset,:]))
# using tsne:
embeddings_tsne = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=50).fit_transform(emb_arr[indices_subset,:])
# using u-map:
embeddings_umap = umap.UMAP(n_components=2).fit_transform(emb_arr[indices_subset, :])


AttributeError: module 'umap' has no attribute 'UMAP'

In [ ]:
################## PLOT TSNE 2-DIM VISUALISATION OF THE EMBEDDING SPACE ##################

cmap = plt.get_cmap('gnuplot2')

# embeddings_tsne=embeddings_tsne[0:500,:]

plt.figure(figsize=(6,6))
plt.suptitle('Visualization of embedding space with TSNE, perplexity=50', fontsize=16)
plt.subplot(3,1,1)
plt.set_cmap(cmap)
plt.scatter(embeddings_tsne[:,0] , embeddings_tsne[:,1], c=rt60_true_arr[indices_subset], s=3);plt.colorbar();plt.title('colormap=RT60')
plt.subplot(3,1,2)
plt.set_cmap(cmap)
plt.scatter(embeddings_tsne[:,0] , embeddings_tsne[:,1], c=edt_meas_arr[indices_subset], s=3);plt.colorbar();plt.title('colormap=EDT')
plt.subplot(3,1,3)
plt.set_cmap(cmap)
plt.scatter(embeddings_tsne[:,0] , embeddings_tsne[:,1], c=c50_meas_arr[indices_subset], s=3);plt.colorbar();plt.title('colormap=C50')
plt.set_cmap(cmap)
plt.tight_layout()
plt.savefig('TSNE.pdf', format='pdf')
plt.show()


plt.figure(figsize=(6,6))
plt.suptitle('Visualization of embedding space with PCA', fontsize=16)
plt.subplot(3,1,1)
plt.scatter(embeddings_pca[:,0] , embeddings_pca[:,1], c=rt60_true_arr[indices_subset], s=3);plt.colorbar();plt.title('colormap=RT60')
plt.subplot(3,1,2)
plt.scatter(embeddings_pca[:,0] , embeddings_pca[:,1], c=edt_meas_arr[indices_subset], s=3);plt.colorbar();plt.title('colormap=EDT')
plt.subplot(3,1,3)
plt.scatter(embeddings_pca[:,0] , embeddings_pca[:,1], c=c50_meas_arr[indices_subset], s=3);plt.colorbar();plt.title('colormap=C50')
plt.tight_layout()
plt.savefig('PCA.pdf', format='pdf')
plt.show()



plt.figure(figsize=(6,6))
plt.suptitle('Visualization of embedding space with UMAP', fontsize=16)
plt.subplot(3,1,1)
plt.scatter(embeddings_umap[:,0] , embeddings_umap[:,1], c=rt60_true_arr[indices_subset], s=3);plt.colorbar();plt.title('colormap=RT60')
plt.subplot(3,1,2)
plt.scatter(embeddings_umap[:,0] , embeddings_umap[:,1], c=edt_meas_arr[indices_subset], s=3);plt.colorbar();plt.title('colormap=EDT')
plt.subplot(3,1,3)
plt.scatter(embeddings_umap[:,0] , embeddings_umap[:,1], c=c50_meas_arr[indices_subset], s=3);plt.colorbar();plt.title('colormap=C50')
plt.tight_layout()
plt.savefig('UMAP.pdf', format='pdf')
plt.show()

In [ ]:
################## GENERATE EMBEDDING ARRAY AND PARAMETER ARRAYS FROM MULTIPLE DATA POINTS FROM THE SAME RIR ##################


from torch.utils.data import Subset



# Create a special data set containing style data with only one specific impulse response
N_audios=100
N_irs=5
emb_arr=np.zeros([N_audios*N_irs,512])
id_array=np.zeros([N_audios*N_irs,1])


# styleIR_indices=list(range(0, N_irs))

styleIR_indices= [0,1,11,3,12]

styleIR_indices=np.random.randint(10000, size=N_irs)

styleIR_indices=[6786, 9377, 2068, 0, 12]

for i,idx in enumerate(styleIR_indices):
    config["split"]="test"
    config["style_rir"]=dataset.get_info(idx)["ir_file_path"]
    print(config["style_rir"])
    dataset_this_ir=ds.DatasetReverbTransfer(config)
    # indices_audios=list(range(0,N_audios))
    indices_audios=list(np.random.randint(10000, size=N_audios))
    dataset_this_ir=Subset(dataset_this_ir,indices_audios)

    for j,data in enumerate(dataset_this_ir):
        # get style audio
        sStyle_in=data[1]
        # get style embedding of size 512
        with torch.no_grad():
            embedding=model.conditioning_network(sStyle_in.unsqueeze(0).to(config["device"]))
            emb_arr[(i*N_audios)+j,:]=embedding.cpu().numpy()
        id_array[(i*N_audios)+j]=i


In [ ]:
styleIR_indices

In [ ]:
################## PERFORM DIMENSIONALITY REDUCTION ON THE EMBEDDING SPACE ##################

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

# reduce dimensionality of the embeddings
# using pca:
embeddings_pca_rir=PCA(n_components=2).fit_transform(emb_arr)
# using first pca and then tsne:
# embeddings_pca_tsne = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=20).fit_transform(PCA(n_components=50).fit_transform(emb_arr))
# using tsne:
embeddings_tsne_rir = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=10).fit_transform(emb_arr)

# using u-map:
embeddings_umap_rir = umap.UMAP(n_components=2).fit_transform(emb_arr)

In [ ]:
a=id_array.astype(str).tolist()

id_array=id_array+1

In [ ]:
################## PLOT TSNE 2-DIM VISUALISATION OF THE EMBEDDING SPACE ##################

# Define a colormap with 5 distinct colors
import matplotlib.colors as mcolors
cmap = mcolors.ListedColormap(['red', 'blue', 'green', 'purple', 'orange'])

plt.figure(figsize=(6,4))
plt.subplot(2,1,1)
plt.suptitle('Visualization of embedding space with TSNE, perplexity=50', fontsize=12)
plt.scatter(embeddings_tsne_rir[:,0] , embeddings_tsne_rir[:,1], c=id_array, cmap=cmap,alpha=0.5);plt.colorbar();plt.title('colormap=RIR ID')
plt.subplot(2,1,2)
plt.suptitle('Visualization of embedding space with PCA', fontsize=12)
plt.scatter(embeddings_pca_rir[:,0] , embeddings_pca_rir[:,1], c=id_array, cmap=cmap, alpha=0.5);plt.colorbar();plt.title('colormap=RIR ID')
plt.tight_layout()
plt.savefig('lat2.pdf', format='pdf')
plt.show()

In [ ]:
# Define a colormap with 5 distinct colors
import matplotlib.colors as mcolors
cmap = mcolors.ListedColormap(['red', 'blue', 'green', 'purple', 'orange'])

plt.figure(figsize=(6,9))
plt.suptitle('Visualization of embedding space with PCA', fontsize=16)
plt.subplot(4,1,1)
plt.scatter(embeddings_pca[:,0] , embeddings_pca[:,1], c=rt60_true_arr[indices_subset], s=3);plt.colorbar();plt.title('colormap=RT60')
plt.subplot(4,1,2)
plt.scatter(embeddings_pca[:,0] , embeddings_pca[:,1], c=edt_meas_arr[indices_subset], s=3);plt.colorbar();plt.title('colormap=EDT')
plt.subplot(4,1,3)
plt.scatter(embeddings_pca[:,0] , embeddings_pca[:,1], c=c50_meas_arr[indices_subset], s=3);plt.colorbar();plt.title('colormap=C50')
plt.subplot(4,1,4)
plt.scatter(embeddings_pca_rir[:,0] , embeddings_pca_rir[:,1], c=id_array, cmap=cmap, alpha=0.5);plt.colorbar();plt.title('colormap=RIR ID')
plt.tight_layout()
plt.savefig('pca.pdf', format='pdf')
plt.show()



plt.figure(figsize=(6,9))
plt.suptitle('Visualization of embedding space with TSNE', fontsize=16)
plt.subplot(4,1,1)
plt.scatter(embeddings_tsne[:,0] , embeddings_tsne[:,1], c=rt60_true_arr[indices_subset], s=3);plt.colorbar();plt.title('colormap=RT60')
plt.subplot(4,1,2)
plt.scatter(embeddings_tsne[:,0] , embeddings_tsne[:,1], c=edt_meas_arr[indices_subset], s=3);plt.colorbar();plt.title('colormap=EDT')
plt.subplot(4,1,3)
plt.scatter(embeddings_tsne[:,0] , embeddings_tsne[:,1], c=c50_meas_arr[indices_subset], s=3);plt.colorbar();plt.title('colormap=C50')
plt.subplot(4,1,4)
plt.scatter(embeddings_tsne_rir[:,0] , embeddings_tsne_rir[:,1], c=id_array, cmap=cmap, alpha=0.5);plt.colorbar();plt.title('colormap=RIR ID')
plt.tight_layout()
plt.savefig('tsne.pdf', format='pdf')
plt.show()


plt.figure(figsize=(6,9))
plt.suptitle('Visualization of embedding space with UMAP', fontsize=16)
plt.subplot(4,1,1)
plt.scatter(embeddings_umap[:,0] , embeddings_umap[:,1], c=rt60_true_arr[indices_subset], s=3);plt.colorbar();plt.title('colormap=RT60')
plt.subplot(4,1,2)
plt.scatter(embeddings_umap[:,0] , embeddings_umap[:,1], c=edt_meas_arr[indices_subset], s=3);plt.colorbar();plt.title('colormap=EDT')
plt.subplot(4,1,3)
plt.scatter(embeddings_umap[:,0] , embeddings_umap[:,1], c=c50_meas_arr[indices_subset], s=3);plt.colorbar();plt.title('colormap=C50')
plt.subplot(4,1,4)
plt.scatter(embeddings_umap_rir[:,0] , embeddings_umap_rir[:,1], c=id_array, cmap=cmap, alpha=0.5);plt.colorbar();plt.title('colormap=RIR ID')
plt.tight_layout()
plt.savefig('umap.pdf', format='pdf')
plt.show()
